# Testing photom step with MIRI simulated data

## Summary

This notebook processes an image through calwebb_image2 and calwebb_image3 (calwebb_detector1 is optional) and examines the output table of the source_catalog step. The steps are as follow:

1) Set up data path and directory and image file name.

2) Modify header information of input simulations (if needed).

3) Run input data through calwebb_detector1.

4) Run output of calwebb_detector1 through the photom step in calwebb_image2.

5) Get photom reference file. 

6) Compare the flat field reference file with the rate/cal image ratio and check that the correct conversion factor was applied.

7) Check that the pixel area information has been attached and that PIXAR_SR, PIXAR_A2, and BUNIT keywords have been populated in the cal image.


The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

Author: T. Temim

### Set up import statements

In [ ]:
import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from astropy.io import fits
import numpy as np
from jwst.datamodels import RampModel, ImageModel, MirImgPhotomModel
from astropy.utils.data import get_pkg_data_filename
from ci_watson.artifactory_helpers import get_bigdata
from astropy import table
import crds

In [ ]:
#import os
#os.environ['CRDS_PATH']='$HOME/crds_cache'
#os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
#os.environ['CRDS_CONTEXT']='jwst_0619.pmap'
#os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'
#export CRDS_SERVER_URL=https://jwst-crds.stsci.edu
#export CRDS_PATH=$HOME/crds_cache
#export CRDS_CONTEXT='jwst_0619.pmap'
#export TEST_BIGDATA=https://bytesalad.stsci.edu/artifactory/

### Print pipeline version number

In [ ]:
jwst.__version__ 

### Read in data from artifactory

In [ ]:
input_file = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'source_catalog',
                     'source_catalog_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits')

### Read in input image as JWST data model

In [ ]:
from jwst import datamodels

im = ImageModel(input_file)

### Modify header information of input simulations (if needed)

In [ ]:
print(im.meta.wcsinfo.wcsaxes)

In [ ]:
im.meta.wcsinfo.wcsaxes=2

In [ ]:
print(im.meta.wcsinfo.wcsaxes)

In [ ]:
del im.meta.wcsinfo.cdelt3
del im.meta.wcsinfo.crpix3
del im.meta.wcsinfo.crval3
del im.meta.wcsinfo.ctype3
del im.meta.wcsinfo.cunit3
del im.meta.wcsinfo.pc3_1
del im.meta.wcsinfo.pc3_2


### Run input data through calwebb_detector1

In [ ]:
# The data read in is an image file already processed through Detector1. This step should only be run 
#if we're starting with a ramp file.

#det1 = Detector1Pipeline()
#det1.save_results = True
#det1.run(im)

### Run output of calwebb_detector1 through calwebb_image2

In [ ]:
#input_file = input_file.replace('rateint.fits', 'rate.fits')

In [ ]:
im2 = Image2Pipeline()
#im2.background.skip = True
im2.assign_wcs.skip = True
im2.flat_field.skip = True
im2.photom.skip=False
im2.resample.skip = True
im2.save_results = True
im2.run(im)

In [ ]:
input_file = input_file.replace('rate.fits', 'cal.fits')

In [ ]:
# read in the calibrated file
im_cal = ImageModel(input_file)

### Calculate the rate/cal image ratio

In [ ]:
ratio_im=im_cal.data/im.data

In [ ]:
print(ratio_im)

### Get photom reference file

In [ ]:
photomreffile = im_cal.meta.ref_file.photom.name
print('Photom reference file', photomreffile)
# find location of file
basename = crds.core.config.pop_crds_uri(photomreffile)
path = crds.locate_file(basename, "jwst")
# open reference file

photom_file = MirImgPhotomModel(path)

In [ ]:
photom_file.meta.photometry.pixelarea_steradians

### Print photom calibration conversion factor for image

In [ ]:
print(photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0])

### Check that same value has been applied to cal image (three values should be the same)

In [ ]:
print('image min','image max','ref file')
print(np.nanmin(ratio_im),np.nanmax(ratio_im),photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0])

### Check that pixel area information has been attached

In [ ]:
im_cal.area
np.nanmin(im_cal.area)
np.nanmax(im_cal.area)

In [ ]:
print('pix_area_min', 'pix_area_max','pix_area_mean')
print(np.nanmin(im_cal.area),np.nanmax(im_cal.area),np.nanmean(im_cal.area))

### Check that PIXAR_SR and PIXAR_A2 keywords have been populated

In [ ]:
im_cal.find_fits_keyword('PIXAR_SR')

In [ ]:
photom_file.find_fits_keyword('PIXAR_SR')

#### Two values should be identical

In [ ]:
print(photom_file.meta.photometry.pixelarea_steradians,im_cal.meta.photometry.pixelarea_steradians)

#### Two values should be identical

In [ ]:
print(photom_file.meta.photometry.pixelarea_arcsecsq, im_cal.meta.photometry.pixelarea_arcsecsq)

### Verify that pixel area in steradians is correct (values should be the same)

In [ ]:
print(im_cal.meta.photometry.pixelarea_arcsecsq/(206265.0**2),im_cal.meta.photometry.pixelarea_steradians)

### Check that bunit information has been added to image ('MJy/sr')

In [ ]:
im_cal.meta.bunit_data

In [ ]:
#photom_file.meta.author